In [311]:
import pandas as pd
import numpy as np
import re

In [312]:
df1 = pd.read_csv("cars-data-table.csv")
df2 = pd.read_csv("auto-data-table-final.csv")
print(df1.head(1))
print(df2.head(1))

   Unnamed: 0  year                                     spec_title fuel_type  \
0           0  2016  2016 Abarth 124 Spider 1.4 MultiAir 16v specs    Petrol   

  transmission     price             body_type number_of_seats  engine_type  \
0       Manual  € 44.760  2-doors, convertible               2  fuel engine   

  fuel_type_detail  ... top_speed  acceleration_0_100_kmh  urban_consumption  \
0         gasoline  ...  232 km/h                   6,8 s        8,5 l/100km   

  extra_urban_consumption combined_consumption co2_emissions battery_range  \
0             5,1 l/100km          6,4 l/100km      148 g/km           NaN   

  battery_capacity charging_time power_consumption  
0              NaN           NaN               NaN  

[1 rows x 26 columns]
   brand model  year                                         spec_title  \
0  Acura    CL  2000  3.2 i V6 24V Type S (263 Hp) Automatic 2000 - ...   

                                            spec_url          fuel_type  \
0  http

In [313]:
def extract_l_per_100km(text):
    if pd.isna(text):
        return None
    # Look for a number followed by 'l/100 km' (ignore case and spaces)
    match = re.search(r'(\d+\.?\d*)\s*l/100\s*km', text, re.IGNORECASE)
    if match:
        return float(match.group(1))
    else:
        return None

In [314]:
print(df1.columns.size)
print(df2.columns.size)

df1.drop(columns=[
    "engine_type",
    "fuel_type_detail",
    "max_power"
], inplace=True)

df1.drop(df1.columns[0], axis=1, inplace=True)

df1.rename(columns={
    "number_of_seats": "seats",
    "total_max_power_hp": "power_hp",
    "total_max_power_kw": "power_kw"
}, inplace=True)

df2.rename(columns={
    "power": "power_hp",
    "fuel_consumption_urban":"urban_consumption",
    "fuel_consumption_extra_urban":"extra_urban_consumption"
}, inplace=True)

print(df1.columns)
print(df1.columns.size)



26
21
Index(['year', 'spec_title', 'fuel_type', 'transmission', 'price', 'body_type',
       'seats', 'power_kw', 'power_hp', 'max_torque', 'fuel_tank_capacity',
       'turbo', 'top_speed', 'acceleration_0_100_kmh', 'urban_consumption',
       'extra_urban_consumption', 'combined_consumption', 'co2_emissions',
       'battery_range', 'battery_capacity', 'charging_time',
       'power_consumption'],
      dtype='object')
22


In [315]:
# Extract number of doors
df1['doors'] = df1['body_type'].str.extract(r'(\d+)-doors')

# Remove the "x-doors, " part from body_type
df1['body_type'] = df1['body_type'].str.replace(r'^\d+-doors,\s*', '', regex=True)

# Optionally convert doors to numeric
df1['doors'] = pd.to_numeric(df1['doors'], errors='coerce')

# Extract just 'yes' or 'no' at the start of the string
df1['turbo'] = df1['turbo'].str.extract(r'^(yes|no)', expand=False)

# Extract the second word (brand) from spec_title
df1['brand'] = df1['spec_title'].str.extract(r'^\d+\s+(\w+)', expand=False)

# Extract the third word (model)
df1['model'] = df1['spec_title'].str.extract(r'^\d+\s+\w+\s+(\w+)', expand=False)

conditions = [
    df1['fuel_type'] == 'Electric',
    df1['turbo'] == 'no',
    df1['turbo'] == 'yes'
]

# Change np.nan to a string representation of missing data
choices = [
    'NaN',  # Or any other string to represent missing data
    'Naturally aspirated engine',
    'Turbocharger, Intercooler'
]

# Also change the default to match the data type in choices
df1['engine_aspiration'] = np.select(conditions, choices, default='NaN')


# Extract horsepower and format with ' Hp', extract rpm number
df2[['power_val', 'rpm']] = df2['power_hp'].str.extract(r'(\d+)\s*Hp\s*@\s*(\d+)\s*rpm', expand=True)

# Create 'power' column with formatted string and keep rpm as number
df2['power_hp'] = df2['power_val'] + ' Hp'
df2['rpm'] = pd.to_numeric(df2['rpm'], errors='coerce')

# Drop the temporary column
df2.drop(columns=['power_val'], inplace=True)

# Apply to both columns
df2['urban_consumption'] = df2['urban_consumption'].apply(extract_l_per_100km)
df2['extra_urban_consumption'] = df2['extra_urban_consumption'].apply(extract_l_per_100km)


# Now compute the average combined_consumption
df2['combined_consumption'] = (
    df2[['urban_consumption', 'extra_urban_consumption']]
    .mean(axis=1)
    .round(2)
    .astype(str) + ' l/100km'
)

df2['urban_consumption'] = df2['urban_consumption'].round(2).astype(str) + ' l/100km'
df2['extra_urban_consumption'] = df2['extra_urban_consumption'].round(2).astype(str) + ' l/100km'

# Step 1: Identify hybrid/electric rows in df2
condition = df2['spec_title'].str.contains('Hybrid', case=False, na=False) | \
            df2['fuel_type'].str.contains('Electricity', case=False, na=False)

# Step 2: Select only needed columns from df1 for the merge
battery_cols = ['brand', 'model', 'year', 'battery_range', 'battery_capacity', 'charging_time', 'power_consumption']
df1_battery = df1[battery_cols]

# Step 3: Merge hybrid/electric rows of df2 with df1 on brand, model, year
df2_hybrid = df2[condition].merge(df1_battery, on=['brand', 'model', 'year'], how='left', suffixes=('', '_from_df1'))

# Step 4: Update the battery-related columns in df2 for hybrid/electric rows
for col in ['battery_range', 'battery_capacity', 'charging_time', 'power_consumption']:
    df2.loc[condition, col] = df2_hybrid[col]

# Optional: If these columns do not exist in df2 yet, create them first with NaNs
for col in ['battery_range', 'battery_capacity', 'charging_time', 'power_consumption']:
    if col not in df2.columns:
        df2[col] = np.nan

print(df1.columns)
print(df1.head(2))
print(df2.head(2))


Index(['year', 'spec_title', 'fuel_type', 'transmission', 'price', 'body_type',
       'seats', 'power_kw', 'power_hp', 'max_torque', 'fuel_tank_capacity',
       'turbo', 'top_speed', 'acceleration_0_100_kmh', 'urban_consumption',
       'extra_urban_consumption', 'combined_consumption', 'co2_emissions',
       'battery_range', 'battery_capacity', 'charging_time',
       'power_consumption', 'doors', 'brand', 'model', 'engine_aspiration'],
      dtype='object')
   year                                     spec_title fuel_type transmission  \
0  2016  2016 Abarth 124 Spider 1.4 MultiAir 16v specs    Petrol       Manual   
1  2016  2016 Abarth 124 Spider 1.4 MultiAir 16v specs    Petrol    Automatic   

      price    body_type seats  power_kw  power_hp max_torque  ...  \
0  € 44.760  convertible     2     125.0     170.0   2500 tpm  ...   
1  € 47.160  convertible     2     125.0     170.0   2500 tpm  ...   

  combined_consumption co2_emissions battery_range battery_capacity  \
0      

In [316]:
for column in df2.columns:
  if column in df1.columns:
    print(f"{column} is in both dataframes")


for column in df2.columns:
  if column not in df1.columns:
    print(f"{column} is not in df2")

brand is in both dataframes
model is in both dataframes
year is in both dataframes
spec_title is in both dataframes
fuel_type is in both dataframes
transmission is in both dataframes
body_type is in both dataframes
seats is in both dataframes
doors is in both dataframes
power_hp is in both dataframes
engine_aspiration is in both dataframes
urban_consumption is in both dataframes
extra_urban_consumption is in both dataframes
combined_consumption is in both dataframes
battery_range is in both dataframes
battery_capacity is in both dataframes
charging_time is in both dataframes
power_consumption is in both dataframes
spec_url is not in df2
engine_modification is not in df2
start_production is not in df2
drive_wheel is not in df2
fuel_injection_system is not in df2
number_of_cylinders is not in df2
engine_configuration is not in df2
tires_size is not in df2
rpm is not in df2


In [317]:
print(len(df1))
print(len(df2))

df1.drop_duplicates(subset=['brand', 'model', 'year', 'spec_title'], keep='first', inplace=True)
df2.drop_duplicates(subset=['brand', 'model', 'year', 'spec_title'], keep='first', inplace=True)

print(len(df1))
print(len(df2))

# Step 1: Concatenate df1 and df2 (aligns columns by name)
combined_df = pd.concat([df1, df2], ignore_index=True)

# Step 2: Drop duplicates based on 'brand', 'model', and 'year'
combined_df.drop_duplicates(subset=['brand', 'model', 'year', 'spec_title'], keep='first', inplace=True)
print(len(combined_df))

combined_df.to_csv("combined_data.csv", index=False)

combined_df.to_json('final_output.json', orient='records', indent=2)


25682
25662
19344
24718
44062


In [318]:
from google.colab import files
files.download('final_output.json')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>